In [ ]:
# Most packages should already be installed in your conda environment
# Run this only if you're missing any packages
# !pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai gradio


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# Imports - Modified for local environment

import os
import time
from io import StringIO
import torch
import numpy as np
import pandas as pd
import random
from openai import OpenAI
from sqlalchemy import create_engine
import gradio as gr
from dotenv import load_dotenv

# Optional: Only import these if you want to use the local Llama model
USE_LOCAL_MODEL = False  # Set to True if you want to use the local Llama model

if USE_LOCAL_MODEL:
    from huggingface_hub import login
    from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [4]:
# Model Constants
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [5]:
# Authentication - Load from .env file
# Make sure you have a .env file in your project root with:
# OPENAI_API_KEY=your-openai-key-here
# HF_TOKEN=your-huggingface-token-here  # Only needed if using local model

# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("Missing OPENAI_API_KEY. Add it to your .env file.")

openai = OpenAI(api_key=openai_api_key)
print("✅ OpenAI client initialized successfully!")

# Optional: Hugging Face authentication (only if using local model)
if USE_LOCAL_MODEL:
    hf_token = os.getenv("HF_TOKEN")
    if not hf_token:
        raise ValueError("Missing HF_TOKEN. Add it to your .env file.")
    login(hf_token, add_to_git_credential=True)
    print("✅ Hugging Face authentication successful!")

✅ OpenAI client initialized successfully!


In [6]:
# Tokenizer Setup (only if using local model)
if USE_LOCAL_MODEL:
    tokenizer = AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token = tokenizer.eos_token

In [7]:
# Model Quantization for Performance Optimization (only if using local model)
if USE_LOCAL_MODEL:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

In [8]:
# Load Model (only if using local model - requires significant GPU memory)
if USE_LOCAL_MODEL:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    if device == "cpu":
        print("Warning: Loading large model on CPU will be very slow!")
    
    model = AutoModelForCausalLM.from_pretrained(
        LLAMA, 
        device_map="auto", 
        quantization_config=quant_config if device == "cuda" else None
    )
else:
    print("Skipping local model loading. Using OpenAI API only.")

Skipping local model loading. Using OpenAI API only.


In [9]:
def generate_ev_driver(num_records, address_type):
    # Adjusting the prompt based on checkbox selection
    address_prompts = {
        "international": f"Generate {num_records} rows of synthetic personal data with international addresses and phone numbers.",
        "us_only": f"Generate {num_records} rows of synthetic personal data with U.S.-only addresses and phone numbers.",
        "us_international": f"Generate {num_records} rows of synthetic personal data with a mix of U.S. and international addresses and phone numbers.",
        "americas": f"Generate {num_records} rows of synthetic personal data with a mix of U.S., Canada, Central America, and South America addresses and phone numbers.",
        "europe": f"Generate {num_records} rows of synthetic personal data with Europe-only addresses and phone numbers.",
    }

    address_prompt = address_prompts.get(address_type, "Generate synthetic personal data.")
    # Generate unique driver IDs
    driver_ids = random.sample(range(1, 1000001), num_records)

    user_prompt = f"""
    {address_prompt}
    Each row should include:
    - driverid (unique from the provided list: {driver_ids})
    - first_name (string)
    - last_name (string)
    - email (string)
    - phone_number (string)
    - address (string)
    - city (string)
    - state (string)
    - zip_code (string)
    - country (string)

    Ensure the CSV format is valid, with proper headers and comma separation.
    """

    response = openai.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates structured CSV data."},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Call the new function to clean and extract the CSV data
    return clean_and_extract_csv(response)

In [10]:
def clean_and_extract_csv(response):
    # Clean up the response and remove the last occurrence of the code block formatting
    csv_data = response.choices[0].message.content.strip()
    csv_data = csv_data.rsplit("```", 1)[0].strip()

    # Define header and split the content to extract the data
    header = "driverid,first_name,last_name,email,phone_number,address,city,state,zip_code,country"
    _, *content = csv_data.split(header, 1)

    # Return the cleaned CSV data along with the header
    return header + content[0].split("\n\n")[0] if content else csv_data

In [11]:
def update_dataset(num_records, address_type):
    response = generate_ev_driver(num_records, address_type)

    # Convert response to DataFrame
    try:
        df = pd.read_csv(StringIO(response))
    except Exception as e:
        return pd.DataFrame(), f"Error parsing dataset: {str(e)}"

    return df, response

In [12]:
# Function to handle address type selection
def check_address_selection(selected_type):
    if not selected_type:
        # Return the error message and set button to yellow and disabled
        return (
            "<span style='color:red;'>⚠️ Address type is required. Please select one.</span>",
            gr.update(interactive=False, elem_classes="yellow_btn")
        )
    # Return success message and set button to blue and enabled
    return (
        "<span style='color:green;'>Ready to generate dataset.</span>",
        gr.update(interactive=True, elem_classes="blue_btn")
    )


In [13]:
# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("## Dynamic CSV Dataset Viewer")

    num_records_slider = gr.Slider(minimum=5, maximum=50, step=5, value=20, label="Number of Records")

    with gr.Row(equal_height=True):
        address_type_radio = gr.Radio(
            ["us_only", "international", "us_international", "americas", "europe"],
            value="",
            label="Address and Phone Type",
            info="Select the type of addresses and phone numbers"
        )
        status_text = gr.Markdown(
            "<span style='color:red;'>⚠️ Please select an address type above to proceed.</span>",
            elem_id="status_text"
        )

    generate_btn = gr.Button("Generate Data", interactive=True, elem_id="generate_btn")

    response_text = gr.Textbox(value="", label="Generated Driver List CSV", interactive=False)
    dataframe_output = gr.Dataframe(value=pd.DataFrame(), label="Generated Driver List Dataset")

    # Update status text and button style dynamically
    address_type_radio.change(fn=check_address_selection, inputs=[address_type_radio], outputs=[status_text, generate_btn])

    generate_btn.click(update_dataset, inputs=[num_records_slider, address_type_radio], outputs=[dataframe_output, response_text])

    # Custom CSS for button colors
    app.css = """
    .blue_btn {
        background-color: green;
        color: white;
    }
    """

app.launch()  # Will open locally in your browser

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
